**Mention that each iteration is just running the model and storing the state, plus some other stuff for recording things etc**

We now demonstrate this concept in action. First, we import the needed packages and set up a simulation (see **link**):

In [1]:
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
import numpy as np

/home/smithis7/anaconda3/envs/tardis/lib/python3.7/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)
/home/smithis7/anaconda3/envs/tardis/lib/python3.7/importlib/_bootstrap.py:219: QAWarning: pyne.material is not yet QA compliant.
  return f(*args, **kwds)


In [2]:
tardis_config = Configuration.from_yaml('tardis_example.yml')
sim = Simulation.from_config(tardis_config)

model = sim.model
plasma = sim.plasma
runner = sim.runner

We show the initial radiative temperature, dilution factor, electron densities, and tau sobolevs in each radial cell (see **link**):

In [3]:
model.t_rad

<Quantity [7705.4185289 , 7693.87838289, 7682.37275176, 7670.9014809 ,
           7659.46441662, 7648.06140616, 7636.69229763, 7625.35694008,
           7614.05518344, 7602.78687852, 7591.55187703, 7580.35003153,
           7569.18119549, 7558.04522319, 7546.94196981, 7535.87129136,
           7524.8330447 , 7513.82708751, 7502.85327833, 7491.91147651] K>

In [4]:
model.w

array([0.40039164, 0.33245223, 0.28966798, 0.2577141 , 0.23224568,
       0.21120466, 0.19341188, 0.17811402, 0.16479446, 0.1530809 ,
       0.14269498, 0.13342262, 0.12509541, 0.11757849, 0.11076215,
       0.10455605, 0.09888494, 0.09368554, 0.08890421, 0.08449514])

In [5]:
plasma.get_value('electron_densities')

0     2.033278e+12
1     2.027521e+12
2     2.021792e+12
3     2.016096e+12
4     2.010432e+12
5     2.004803e+12
6     1.999211e+12
7     1.993657e+12
8     1.988143e+12
9     1.982671e+12
10    1.977241e+12
11    1.971856e+12
12    1.966517e+12
13    1.961224e+12
14    1.955980e+12
15    1.950786e+12
16    1.945642e+12
17    1.940551e+12
18    1.935512e+12
19    1.930528e+12
dtype: float64

In [7]:
plasma.get_value('tau_sobolevs')

We set the number of packets and we run one iteration of the Monte Carlo simulation:

In [ ]:
N_packets = 10000

runner.run(model, plasma, N_packets)

We now show the values of the three estimators previously mentioned:

In [ ]:
runner.j_estimator

In [ ]:
runner.nu_bar_estimator

In [ ]:
# Because most rows of the j_blue estimatior are partially or mostly
# zero, we show just rows with all nonzero elements
runner.j_blue_estimator[np.all(runner.j_blue_estimator != 0, axis=1)]

We note that the shape of the j_blue estimator and the tau_sobolevs are the same: namely, each contain a value for each possible atomic line transition in each radial cell (as opposed to the other two estimators which just have one value for each cell):

In [ ]:
plasma.get_value('tau_sobolevs').shape, runner.j_blue_estimator.shape

We now advance the state of the simulation based on the estimators, and demonstrate this by showing the four quantities we showed before running the simulation. Compare them with their values above!

In [ ]:
# When advance_state is called, a brief summary of the updated t_rad's
# and w's is given
sim.advance_state();

In [ ]:
model.t_rad

In [ ]:
model.w

In [ ]:
plasma.get_value('electron_densities')

In [ ]:
plasma.get_value('tau_sobolevs')